In [0]:
import meteo
import scipy.optimize as opt
from IPython.html.widgets import interact
import seaborn as sns
sns.set_style('white')

In [0]:
mod = meteo.Model('models/model.json', constants={'alpha': 3.0})

In [0]:
(t_path,par_path,var_path) = mod.homotopy_bde({"z": 2.0},{"z": 3.0},{"x": 1.0},plot=True); sns.despine()
#(t_path,par_path,var_path) = mod.homotopy_bde({"z": 2.0},{"z": 3.0},{"x": 1.0},plot=True)

In [0]:
var_path[:,0]

In [0]:
mod_de = meteo.Model('models/model_diffeq.json')

In [0]:
alpha0 = 1.0
alpha1 = 2.0

xgrid = mod_de.arg_info['x']['grid']
fgrid = np.exp(alpha0*xgrid)
fpgrid = alpha0*np.exp(alpha0*xgrid)

(t_path,par_path,var_path) = mod_de.homotopy_bde({"alpha":alpha0},{"alpha":alpha1},{"f":[fgrid,fpgrid]},delt=0.05)
varf_dict = mod_de.array_to_dict(var_path[-1],mod_de.var_info,mod_de.var_sizes)

In [0]:
grid = mod_de.arg_info['x']['grid']
(f_fin,fp_fin) = varf_dict['f']
f_theor = np.exp(alpha1*grid)
(fig,(ax1,ax2)) = subplots(1,2,figsize=(14,5))
ax1.plot(grid,f_theor,grid,f_fin); sns.despine(ax=ax1);
ax2.plot(grid,np.log(np.abs((f_fin-f_theor)/f_theor))); sns.despine(ax=ax2);

In [0]:
var_val = mod.solve_system({"z": 2.0},{"x": 1.1},output=True)

In [0]:
mod_de2 = meteo.Model('models/model_diffeq2.json')

In [0]:
pi = mod_de2.con_dict['pi']
r = mod_de2.con_dict['r']
gamma = mod_de2.con_dict['gamma']
c = mod_de2.con_dict['c']
qgrid = mod_de2.arg_info['q']['grid']

Wmin = gamma*c*r**(gamma-1)
Wmax = 20.0
Wfunc = lambda W: r*W-pi-(gamma-1)*c*(W/(gamma*c))**(gamma/(gamma-1))
Wbar = opt.brentq(Wfunc,Wmin,Wmax)
print Wbar,Wfunc(Wbar)

Wgrid = Wbar*qgrid
Wpgrid = Wbar*np.ones_like(qgrid)
xgrid = (Wbar/(gamma*c))**(1/(gamma-1))*np.ones_like(qgrid)

(t_path,par_path,var_path) = mod_de2.homotopy_bde({"tau":0.0,"g":0.0},{"tau":0.15,"g":0.02},
                                                  {"Wbar":Wbar,"W":[Wgrid,Wpgrid],"x":xgrid},
                                                  solve=False,output=False,delt=0.2,max_step=1000,max_newton=10)

parf_dict = mod_de2.array_to_dict(par_path[-1],mod_de2.par_info,mod_de2.par_sizes)
varf_dict = mod_de2.array_to_dict(var_path[-1],mod_de2.var_info,mod_de2.var_sizes)

In [0]:
print 'Wbar = {}'.format(Wbar)
print 'Max err = {}'.format(np.max(np.abs(mod_de2.eval_system(parf_dict,varf_dict))))
grid = mod_de2.arg_info['q']['grid']
(W_fin,dW_fin,ddW_fin) = varf_dict['W']
x_fin = varf_dict['x']
(fig,(ax1,ax2)) = subplots(1,2,figsize=(14,5))
ax1.plot(grid,dW_fin); sns.despine(ax=ax1);
ax2.plot(grid,x_fin); sns.despine(ax=ax2);

In [0]:
mod_de3 = meteo.Model('models/model_diffeq3.json')

In [0]:
tau = 0.1
g = 0.015
pi = mod_de3.con_dict['pi']
r = mod_de3.con_dict['r']
gamma = mod_de3.con_dict['gamma']
c = mod_de3.con_dict['c']
nu = mod_de3.con_dict['nu']
qgrid = mod_de3.arg_info['q']['grid']

Wmin = 0.0
Wmax = gamma*c*r**(gamma-1)
Wfunc = lambda W: r*W-pi-(gamma-1)*c*(W/(gamma*c))**(gamma/(gamma-1))
#Wbar = opt.brentq(Wfunc,Wmin,Wmax)
Wbar = opt.fsolve(Wfunc,1.0)[0]
xmax = (Wbar/(c*gamma))**(1/(gamma-1))
print 'Wbar = {}, Werr = {}, xmax = {}'.format(Wbar,Wfunc(Wbar),xmax)
print

Wgrid = Wbar*qgrid
Wpgrid = Wbar*np.ones_like(qgrid)
xgrid = (Wbar/(gamma*c))**(1/(gamma-1))*np.ones_like(qgrid)

(t_path,par_path,var_path) = mod_de3.homotopy_bde({"tau":0.0,"g":0.0},{"tau":tau,"g":g},
                                                  {"Wbar":Wbar,"W":[Wgrid,Wpgrid],"x":xgrid},
                                                  solve=False,output=True,delt=0.01,max_step=1000,max_newton=10)

t_path_good = t_path[::2]
par_path_good = par_path[::2]
var_path_good = var_path[::2]

In [0]:
def plot_index(idx=0,icut1=0,icut2=256):
    parf_dict = mod_de3.array_to_dict(par_path_good[idx],mod_de3.par_info,mod_de3.par_sizes)
    varf_dict = mod_de3.array_to_dict(var_path_good[idx],mod_de3.var_info,mod_de3.var_sizes)

    tau = parf_dict['tau']
    g = parf_dict['g']
    
    print 'Max err = {}'.format(np.max(np.abs(mod_de3.eval_system(parf_dict,varf_dict))))
    print 'g = {}, tau = {}'.format(parf_dict['g'],parf_dict['tau'])

    grid = mod_de3.arg_info['q']['grid']
    (W_fin,dW_fin) = varf_dict['W']
    x_fin = varf_dict['x']
    
    Ediff = (qgrid/(4*c))*dW_fin**2 - g*grid*dW_fin + (pi*grid-(r-g+tau*(1-grid))*W_fin)
    W_lin = grid*(pi-c*g**2)/(r-g+tau*(1-grid))
    dW_lin = (pi-c*g**2)/(r-g+tau*(1-grid))*(1+grid*tau/(r-g+tau*(1-grid)))
    S_fin = W_fin/W_lin
    print S_fin[1]

    (fig,(ax1,ax2)) = subplots(1,2,figsize=(10,4))
    ax1.plot(grid[icut1:icut2],S_fin[icut1:icut2]); ax1.set_ylim(1.0,1.8); sns.despine(ax=ax1);
    ax2.plot(grid[icut1:icut2],x_fin[icut1:icut2]); ax2.set_ylim(0.0,0.07) sns.despine(ax=ax2);
    
interact(plot_index,idx=(0,len(t_path_good)-1),icut1=(0,256),icut2=(0,256));

In [0]:
mod_eq = meteo.Model('models/frontier_equil.json')

In [0]:
g = 0.02
delta = mod_eq.con_dict['delta']
gamma = mod_eq.con_dict['gamma']
c = mod_eq.con_dict['c']
d = mod_eq.con_dict['d']
nu = mod_eq.con_dict['nu']
qgrid = mod_eq.arg_info['q']['grid']

Wmin = 0.0
Wmax = gamma*c*delta**(gamma-1)
Wfunc = lambda W: delta*W-1.0-(gamma-1)*c*(W/(gamma*c))**(gamma/(gamma-1))
#Wbar = opt.brentq(Wfunc,Wmin,Wmax)
Wbar = opt.fsolve(Wfunc,1.0)[0]
xmax = (Wbar/(c*gamma))**(1/(gamma-1))
print 'Wbar = {}, Werr = {}, xmax = {}'.format(Wbar,Wfunc(Wbar),xmax)
print

# initial values
Wgrid = Wbar*qgrid
Wpgrid = Wbar*np.ones_like(qgrid)
Zgrid = Wbar*(nu/(nu+1))*(1-qgrid**(nu+1))
Zpgrid = -Wbar*nu*qgrid**nu
xgrid = (Wbar/(gamma*c))**(1/(gamma-1))*np.ones_like(qgrid)
ygrid = (Zgrid/(gamma*d))**(1/(gamma-1))*np.ones_like(qgrid)

(t_path_eq,par_path_eq,var_path_eq) = mod_eq.homotopy_bde({"alpha":0.0,"g":0.0},{"alpha":1.0,"g":g},
                                                  {"Wbar":Wbar,"W":[Wgrid,Wpgrid],"Z":[Zgrid,Zpgrid],"x":xgrid,"y":ygrid},
                                                  solve=True,output=True,delt=0.05,max_step=1000,max_newton=10)

t_path_good_eq = t_path_eq[::2]
par_path_good_eq = par_path_eq[::2]
var_path_good_eq = var_path_eq[::2]

In [0]:
def plot_index(idx=0):
    parf_dict = mod_eq.array_to_dict(par_path_good_eq[idx],mod_eq.par_info,mod_eq.par_sizes)
    varf_dict = mod_eq.array_to_dict(var_path_good_eq[idx],mod_eq.var_info,mod_eq.var_sizes)

    print 'Max err = {}'.format(np.max(np.abs(mod_eq.eval_system(parf_dict,varf_dict))))

    grid = mod_eq.arg_info['q']['grid']
    (W_fin,dW_fin) = varf_dict['W']
    x_fin = varf_dict['x']
    y_fin = varf_dict['y']

    (fig,(ax1,ax2)) = subplots(1,2,figsize=(10,4))
    ax1.plot(grid,x_fin); sns.despine(ax=ax1);
    ax2.plot(grid,y_fin); sns.despine(ax=ax2);

interact(plot_index,idx=(0,len(t_path_good_eq)-1));

In [0]:
mod_sp = meteo.Model('models/frontier_socplan.json')

In [0]:
g = 0.02
delta = mod_sp.con_dict['delta']
gamma = mod_sp.con_dict['gamma']
c = mod_sp.con_dict['c']
d = mod_sp.con_dict['d']
nu = mod_sp.con_dict['nu']
qgrid = mod_sp.arg_info['q']['grid']

Wmin = 0.0
Wmax = gamma*c*delta**(gamma-1)
Wfunc = lambda W: delta*W-1.0-(gamma-1)*c*(W/(gamma*c))**(gamma/(gamma-1))
#Wbar = opt.brentq(Wfunc,Wmin,Wmax)
Wbar = opt.fsolve(Wfunc,1.0)[0]
xmax = (Wbar/(c*gamma))**(1/(gamma-1))
print 'Wbar = {}, Werr = {}, xmax = {}'.format(Wbar,Wfunc(Wbar),xmax)
print

# initial values
Wgrid = Wbar*qgrid
Wpgrid = Wbar*np.ones_like(qgrid)
Zgrid = Wbar*(nu/(nu+1))*(1-qgrid**(nu+1))
Zpgrid = -Wbar*nu*qgrid**nu
xgrid = (Wbar/(gamma*c))**(1/(gamma-1))*np.ones_like(qgrid)
ygrid = ((Zgrid-(1-qgrid**nu)*Wgrid)/(gamma*d))**(1/(gamma-1))*np.ones_like(qgrid)

(t_path_sp,par_path_sp,var_path_sp) = mod_sp.homotopy_bde({"alpha":0.0,"g":0.0},{"alpha":1.0,"g":g},
                                                  {"Wbar":Wbar,"W":[Wgrid,Wpgrid],"Z":[Zgrid,Zpgrid],"x":xgrid,"y":ygrid},
                                                  solve=True,output=True,delt=0.05,max_step=1000,max_newton=10)

t_path_good_sp = t_path_sp[::2]
par_path_good_sp = par_path_sp[::2]
var_path_good_sp = var_path_sp[::2]

In [0]:
def plot_index(idx=0):
    parf_dict = mod_sp.array_to_dict(par_path_good_sp[idx],mod_sp.par_info,mod_sp.par_sizes)
    varf_dict = mod_sp.array_to_dict(var_path_good_sp[idx],mod_sp.var_info,mod_sp.var_sizes)

    print 'Max err = {}'.format(np.max(np.abs(mod_sp.eval_system(parf_dict,varf_dict))))

    grid = mod_eq.arg_info['q']['grid']
    (W_fin,dW_fin) = varf_dict['W']
    x_fin = varf_dict['x']
    y_fin = varf_dict['y']

    (fig,(ax1,ax2)) = subplots(1,2,figsize=(10,4))
    ax1.plot(grid,x_fin); sns.despine(ax=ax1);
    ax2.plot(grid,y_fin); sns.despine(ax=ax2);

interact(plot_index,idx=(0,len(t_path_good_sp)-1));

In [0]:
# compare decentralized equilibrium and social planner's optimum

grid = mod_eq.arg_info['q']['grid']

parf_dict_eq = mod_eq.array_to_dict(par_path_good_eq[-1],mod_eq.par_info,mod_eq.par_sizes)
varf_dict_eq = mod_eq.array_to_dict(var_path_good_eq[-1],mod_eq.var_info,mod_eq.var_sizes)

parf_dict_sp = mod_sp.array_to_dict(par_path_good_sp[-1],mod_sp.par_info,mod_sp.par_sizes)
varf_dict_sp = mod_sp.array_to_dict(var_path_good_sp[-1],mod_sp.var_info,mod_sp.var_sizes)

x_eq = varf_dict_eq['x']
y_eq = varf_dict_eq['y']

x_sp = varf_dict_sp['x']
y_sp = varf_dict_sp['y']

(fig,(ax1,ax2)) = subplots(1,2,figsize=(10,4))
ax1.plot(grid,x_eq,grid,x_sp); sns.despine(ax=ax1);
ax2.plot(grid,y_eq,grid,y_sp); sns.despine(ax=ax2);

In [0]:
mod_po = meteo.Model('models/frontier_privopt.json',constants={'x': x_sp,'y': y_sp})

In [0]:
g = 0.02
delta = mod_po.con_dict['delta']
gamma = mod_po.con_dict['gamma']
c = mod_po.con_dict['c']
d = mod_po.con_dict['d']
nu = mod_po.con_dict['nu']
qgrid = mod_po.arg_info['q']['grid']

Wmin = 0.0
Wmax = gamma*c*delta**(gamma-1)
Wfunc = lambda W: delta*W-1.0-(gamma-1)*c*(W/(gamma*c))**(gamma/(gamma-1))
#Wbar = opt.brentq(Wfunc,Wmin,Wmax)
Wbar = opt.fsolve(Wfunc,1.0)[0]
x0 = (Wbar/(c*gamma))**(1/(gamma-1))
print 'Wbar = {}, Werr = {}, xmax = {}'.format(Wbar,Wfunc(Wbar),xmax)
print

# initial values
Wgrid = Wbar*qgrid
Wpgrid = Wbar*np.ones_like(qgrid)
Zgrid = Wbar*(nu/(nu+1))*(1-qgrid**(nu+1))
Zpgrid = -Wbar*nu*qgrid**nu

(t_path_po,par_path_po,var_path_po) = mod_po.homotopy_bde({"alpha":0.0,"g":0.0},{"alpha":1.0,"g":g},
                                                  {"Wbar":Wbar,"x0":x0,"W":[Wgrid,Wpgrid],"Z":[Zgrid,Zpgrid]},
                                                  solve=True,output=True,delt=0.025,max_step=1000,max_newton=10)

t_path_good_po = t_path_po[::2]
par_path_good_po = par_path_po[::2]
var_path_good_po = var_path_po[::2]